In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from torch.autograd import Variable
%matplotlib inline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

np.random.seed(1234)
torch.manual_seed(1234)

In [6]:
learn_data = pd.read_csv("churn_modelling_audience.csv")
pd.options.display.max_columns = None
learn_data

,id,last_order_city_name,customer_type,last_order_device_channel,days_from_last_order,days_from_last_seen_at,lifetime_order_count,last30_days_order_amount,last30_days_order_count,last90_days_order_amount,last90_days_order_count,last365_days_order_amount,last365_days_order_count,churn
0,20002006408704,İstanbul,INDIVIDUAL,BROWSER,149,125,1,0,0,0,0,6375,1,True
1,20002005920692,Artvin,INDIVIDUAL,BROWSER,190,58,1,0,0,0,0,18435,1,True
2,20002001678732,İzmir,INDIVIDUAL,APP,89,88,1,0,0,6086,1,6086,1,False
3,20002002795327,Hatay,INDIVIDUAL,APP,162,58,1,0,0,0,0,10615,1,True
4,20002006408404,Bursa,INDIVIDUAL,BROWSER,149,89,1,0,0,0,0,15186,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,20002003647460,Ankara,INDIVIDUAL,BROWSER,358,264,3,0,0,0,0,26996,1,True
15996,20002009472837,Ankara,INDIVIDUAL,APP,46,46,1,0,0,6058,1,6058,1,False
15997,20002009066319,Ankara,INDIVIDUAL,APP,78,15,1,0,0,11934,1,11934,1,False
15998,20002003401739,Ankara,INDIVIDUAL,APP,341,170,5,0,0,0,0,37651,2,True


In [7]:
numerical_columns = ['days_from_last_order', 'days_from_last_seen_at', 'lifetime_order_count',
                     'last30_days_order_amount', 'last30_days_order_count','last90_days_order_amount','last90_days_order_count','last365_days_order_amount','last365_days_order_count']

categorical_columns = ['last_order_city_name', 'customer_type', 'last_order_device_channel','churn']
outputs = ['churn']

In [8]:
churn_data = learn_data[learn_data['churn'] == 'true']
notchurn_data = learn_data[learn_data['churn'] == 'false']

In [9]:
numerical_data = np.stack([learn_data[col].values for col in numerical_columns], 1)
numerical_data = torch.tensor(numerical_data, dtype=torch.float)

for category in categorical_columns:
    learn_data[category] = learn_data[category].astype('category')
    
cn = learn_data['last_order_city_name'].cat.codes.values
ct = learn_data['customer_type'].cat.codes.values
dc = learn_data['last_order_device_channel'].cat.codes.values

categorical_data = np.stack([cn,ct,dc], 1)
categorical_data = torch.tensor(categorical_data, dtype=torch.int64)

learn_data[outputs] = learn_data[outputs].astype(int)

outputs = torch.tensor(learn_data[outputs].values).flatten()
outputs = outputs.long()

In [10]:
total_records = 16000
train_records = int(total_records * .6)
valid_records = int(total_records * .2)
test_records = int(total_records * .2)
numerical_train_data = numerical_data[:train_records]
numerical_valid_data = numerical_data[train_records:train_records+valid_records]
numerical_test_data = numerical_data[train_records+valid_records:total_records]
categorical_train_data = categorical_data[:train_records]
categorical_valid_data = categorical_data[train_records:train_records+valid_records]
categorical_test_data = categorical_data[train_records+valid_records:total_records]
train_outputs = outputs[:train_records]
valid_outputs = outputs[train_records:train_records+valid_records]
test_outputs = outputs[train_records+valid_records:total_records]